<a href="https://colab.research.google.com/github/BeatrizMalengo/builderscase/blob/main/dim_tempo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install google-cloud-storage
from pydrive.drive import GoogleDrive
from google.cloud import storage
import pandas as pd

In [25]:
# Função para mapear os nomes dos dias da semana em português, porque parece que a configuração do locale "pt_BR.UTF-8" não é suportada no ambiente atual do Colab >:(
def obter_nome_dia_semana(data):
    dias_semana = ['segunda-feira', 'terça-feira', 'quarta-feira', 'quinta-feira', 'sexta-feira', 'sábado', 'domingo']
    return dias_semana[data.weekday()]

# Função para mapear os nomes dos meses em português, porque parece que a configuração do locale "pt_BR.UTF-8" não é suportada no ambiente atual do Colab >:(
def obter_nome_mes(data):
    meses = ['janeiro', 'fevereiro', 'março', 'abril', 'maio', 'junho', 'julho', 'agosto', 'setembro', 'outubro', 'novembro', 'dezembro']
    return meses[data.month - 1]

# Criação do DataFrame da dimensão de tempo
start_date = '01/01/2020'
end_date = '31/12/2022'

# Especifica o formato das datas para análise consistente
date_format = "%d/%m/%Y"

# Cria o range de datas com o formato especificado
periodos = pd.date_range(start=pd.to_datetime(start_date, format=date_format),
                         end=pd.to_datetime(end_date, format=date_format),
                         freq='D')

# Criação do DataFrame da dimensão de tempo
df_dim_tempo = pd.DataFrame({'data': periodos})

# Adiciona as informações de período ao DataFrame
df_dim_tempo['data'] = periodos
df_dim_tempo['ano'] = df_dim_tempo['data'].dt.year
df_dim_tempo['mes'] = df_dim_tempo['data'].dt.month
df_dim_tempo['nome_mes'] = df_dim_tempo['data'].apply(obter_nome_mes)
df_dim_tempo['semana_ano'] = df_dim_tempo['data'].dt.week
df_dim_tempo['dia_semana'] = df_dim_tempo['data'].apply(obter_nome_dia_semana)
df_dim_tempo['trimestre'] = df_dim_tempo['data'].dt.quarter
df_dim_tempo['semestre'] = (df_dim_tempo['mes'] - 1) // 6 + 1

df_dim_tempo

<ipython-input-25-1c5250a3f4b2>:31: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df_dim_tempo['semana_ano'] = df_dim_tempo['data'].dt.week


,data,ano,mes,nome_mes,semana_ano,dia_semana,trimestre,semestre
0,2020-01-01,2020,1,janeiro,1,quarta-feira,1,1
1,2020-01-02,2020,1,janeiro,1,quinta-feira,1,1
2,2020-01-03,2020,1,janeiro,1,sexta-feira,1,1
3,2020-01-04,2020,1,janeiro,1,sábado,1,1
4,2020-01-05,2020,1,janeiro,1,domingo,1,1
...,...,...,...,...,...,...,...,...
1091,2022-12-27,2022,12,dezembro,52,terça-feira,4,2
1092,2022-12-28,2022,12,dezembro,52,quarta-feira,4,2
1093,2022-12-29,2022,12,dezembro,52,quinta-feira,4,2
1094,2022-12-30,2022,12,dezembro,52,sexta-feira,4,2


In [23]:
# Define o caminho do arquivo JSON de credenciais
credentials_path = '/content/drive/MyDrive/case-builders-1ff59ff8f179.json'

# Criar o cliente do Cloud Storage
storage_client = storage.Client.from_service_account_json(credentials_path)

# Salvar o DataFrame como arquivo CSV no bucket do Google Cloud Storage
bucket_name = 'bucket_case-builders'
file_name = 'dim_tempo.csv'
bucket = storage_client.get_bucket(bucket_name)
blob = bucket.blob(file_name)
blob.upload_from_string(df_dim_tempo.to_csv(index=False), 'text/csv')

print("Tabela de dimensão de tempo salva com sucesso!")

Tabela de dimensão de tempo salva com sucesso!
